# Retrieval Augmented Question & Answering with Amazon Bedrock using LangChain

### Challenges

When trying to solve a Question Answering task over a larger document corpus with the help of LLMs we need to master the following challenges (amongst others):
- How to manage large document(s) that exceed the token limit
- How to find the document(s) relevant to the question being asked

### Infusing knowledge into LLM-powered systems

We have two primary types of knowledge for LLMs:
- **Parametric knowledge**: refers to everything the LLM learned during training and acts as a frozen snapshot of the world for the LLM. 
- **Source knowledge**: covers any information fed into the LLM via the input prompt. 

When trying to infuse knowledge into a generative AI - powered application we need to choose which of these types to target. Fine-tuning, explored in other workshops, deals with elevating the parametric knowledge through fine-tuning. Since fine-tuning is a resouce intensive operation, this option is well suited for infusing static domain-specific information like domain-specific langauage/writing styles (medical domain, science domain, ...) or optimizing performance towards a very specific task (classification, sentiment analysis, RLHF, instruction-finetuning, ...). 

In contrast to that, targeting the source knowledge for domain-specific performance uplift is very well suited for all kinds of dynamic information, from knowledge bases in structured and unstructured form up to integration of information from live systems. This Lab is about retrieval-augmented generation, a common design pattern for ingesting domain-specific information through the source knowledge. It is particularily well suited for ingestion of information in form of unstructured text with semi-frequent update cycles. 

In this notebook we explain how to utilize the RAG (retrieval-agumented generation) pattern originating from [this](https://arxiv.org/pdf/2005.11401.pdf) paper published by Lewis et al in 2021. It is particularily useful for Question Answering by finding and leveraging the most useful excerpts of documents out of a larger document corpus providing answers to the user questions.

#### Prepare documents
![Embeddings](https://singlestoredevday.s3.amazonaws.com/images/RAG_pipeline_S2_Bedrock.png)

Before being able to answer the questions, the documents must be processed and a stored in a document store index
- Load the documents
- Process and split them into smaller chunks
- Create a numerical vector representation of each chunk using Amazon Bedrock Titan Embeddings model
- Create an index using the chunks and the corresponding embeddings
#### Ask question
![Question](https://singlestoredevday.s3.amazonaws.com/images/RAG_runtime_S2_Bedrock.png)

When the documents index is prepared, you are ready to ask the questions and relevant documents will be fetched based on the question being asked. Following steps will be executed.
- Create an embedding of the input question
- Compare the question embedding with the embeddings in the index
- Fetch the (top N) relevant document chunks
- Add those chunks as part of the context in the prompt
- Send the prompt to the model under Amazon Bedrock
- Get the contextual answer based on the documents retrieved

## Usecase
#### Dataset
In this example, you will use several years of Amazon's Letter to Shareholders as a text corpus to perform Q&A on.

## Implementation
In order to follow the RAG approach this notebook is using the LangChain framework where it has integrations with different services and tools that allow efficient building of patterns such as RAG. We will be using the following tools:

- **LLM (Large Language Model)**: Anthropic Claude available through Amazon Bedrock

  This model will be used to understand the document chunks and provide an answer in human friendly manner.
- **Embeddings Model**: Amazon Titan Embeddings available through Amazon Bedrock

  This model will be used to generate a numerical representation of the textual documents

- **Document Loader**: 
    - PDF Loader available through LangChain for PDFs

  These are loaders that can load the documents from a source, for the sake of this notebook we are loading the sample files from a local path. This could easily be replaced with a loader to load documents from enterprise internal systems.

- **Vector Store**: SingleStoreDB available through LangChain
  In this notebook we are using SingleStoreDB to store both the embeddings and the documents.

Then begin with instantiating the LLM and the Embeddings model. Here we are using Anthropic Claude to demonstrate the use case.

Note: It is possible to choose other models available with Bedrock. You can replace the `model_id` as follows to change the model.

`llm = Bedrock(model_id="...")`

In [4]:
!pip install boto3 langchain pypdf tiktoken sqlalchemy --upgrade --quiet

In [5]:
import getpass

os.environ['AWS_DEFAULT_REGION']='us-east-1'
os.environ['AWS_ACCESS_KEY_ID']= getpass.getpass("AWS_ACCESS_KEY_ID: ")
os.environ['AWS_SECRET_ACCESS_KEY']=getpass.getpass("AWS_SECRET_ACCESS_KEY: ")

AWS_ACCESS_KEY_ID:  ········
AWS_SECRET_ACCESS_KEY:  ········


In [9]:
import boto3
import json
import sys

In [10]:
from io import StringIO
import sys
import textwrap


def print_ww(*args, width: int = 100, **kwargs):
    """Like print(), but wraps output to `width` characters (default 100)"""
    buffer = StringIO()
    try:
        _stdout = sys.stdout
        sys.stdout = buffer
        print(*args, **kwargs)
        output = buffer.getvalue()
    finally:
        sys.stdout = _stdout
    for line in output.splitlines():
        print("\n".join(textwrap.wrap(line, width=width)))

In [11]:
session = boto3.session.Session()

In [12]:
bedrock_client = session.client('bedrock')
bedrock_client.list_foundation_models()

{'ResponseMetadata': {'RequestId': 'f6b7f65d-e385-4125-a0cc-181f3f1bcec5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 13 Nov 2023 21:34:51 GMT',
   'content-type': 'application/json',
   'content-length': '8260',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'f6b7f65d-e385-4125-a0cc-181f3f1bcec5'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large',
   'modelName': 'Titan Text Large',
   'providerName': 'Amazon',
   'inputModalities': ['TEXT'],
   'outputModalities': ['TEXT'],
   'responseStreamingSupported': True,
   'customizationsSupported': ['FINE_TUNING'],
   'inferenceTypesSupported': ['ON_DEMAND']},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-embed-g1-text-02',
   'modelId': 'amazon.titan-embed-g1-text-02',
   'modelName': 'Titan Text Embeddings v2',
   'providerName': 'Amazon',
   'inputModalities': ['TEXT'],
   'ou

### Setup langchain

We create an instance of the Bedrock classes for the LLM and the embedding models. At the time of writing, Bedrock supports one embedding model and therefore we do not need to specify any model id. To be able to compare token consumption across the different RAG-approaches shown in the workshop labs we use langchain callbacks to count token consumption.

In [13]:
bedrock_runtime_client = session.client('bedrock-runtime')
# We will be using the Titan Embeddings Model to generate our Embeddings.
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# - create the Anthropic Model
llm = Bedrock(model_id="anthropic.claude-v2", 
              client=bedrock_runtime_client, 
              model_kwargs={
                  'max_tokens_to_sample': 200
              }, 
              callbacks=[StreamingStdOutCallbackHandler()])

# - create the Titan Embeddings Model
bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1",
                                       client=bedrock_runtime_client)

### Data Preparation
Let's first download some of the files to build our document store.

In this example, you will use several years of Amazon's Letter to Shareholders as a text corpus to perform Q&A on.

In [14]:
!mkdir -p ./RAG_Titan_Embeddings_Claude_data

from urllib.request import urlretrieve
urls = [
    'https://s2.q4cdn.com/299287126/files/doc_financials/2023/ar/2022-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2022/ar/2021-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2021/ar/Amazon-2020-Shareholder-Letter-and-1997-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2020/ar/2019-Shareholder-Letter.pdf'
]

filenames = [
    'AMZN-2022-Shareholder-Letter.pdf',
    'AMZN-2021-Shareholder-Letter.pdf',
    'AMZN-2020-Shareholder-Letter.pdf',
    'AMZN-2019-Shareholder-Letter.pdf'
]

metadata = [
    dict(year=2022, source=filenames[0]),
    dict(year=2021, source=filenames[1]),
    dict(year=2020, source=filenames[2]),
    dict(year=2019, source=filenames[3])]

data_root = "./RAG_Titan_Embeddings_Claude_data/"

for idx, url in enumerate(urls):
    file_path = data_root + filenames[idx]
    urlretrieve(url, file_path)

As part of Amazon's culture, the CEO always includes a copy of the 1997 Letter to Shareholders with every new release. This will cause repetition, take longer to generate embeddings, and may skew your results. In the next section you will take the downloaded data, trim the 1997 letter (last 3 pages) and overwrite them as processed files.

In [15]:
from pypdf import PdfReader, PdfWriter
import glob

local_pdfs = glob.glob(data_root + '*.pdf')

for local_pdf in local_pdfs:
    pdf_reader = PdfReader(local_pdf)
    pdf_writer = PdfWriter()
    for pagenum in range(len(pdf_reader.pages)-3):
        page = pdf_reader.pages[pagenum]
        pdf_writer.add_page(page)

    with open(local_pdf, 'wb') as new_file:
        new_file.seek(0)
        pdf_writer.write(new_file)
        new_file.truncate()

After downloading we can load the documents with the help of [DirectoryLoader from PyPDF available under LangChain](https://python.langchain.com/en/latest/reference/modules/document_loaders.html) and splitting them into smaller chunks.

Note: The retrieved document/text should be large enough to contain enough information to answer a question; but small enough to fit into the LLM prompt. Also the embeddings model has a limit of the length of input tokens limited to 512 tokens, which roughly translates to ~2000 characters. For the sake of this use-case we are creating chunks of roughly 1000 characters with an overlap of 100 characters using [RecursiveCharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/recursive_text_splitter.html).

In [16]:
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

documents = []

for idx, file in enumerate(filenames):
    loader = PyPDFLoader(data_root + file)
    document = loader.load()
    for document_fragment in document:
        document_fragment.metadata = metadata[idx]
        
    #print(f'{len(document)} {document}\n')
    documents += document

# - in our testing Character split works better with this PDF data set
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1000,
    chunk_overlap  = 100,
)

docs = text_splitter.split_documents(documents)

Before we are proceeding we are looking into some interesting statistics regarding the document preprocessing we just performed:

In [17]:
avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in documents])//len(documents)
print(f'Average length among {len(documents)} documents loaded is {avg_doc_length(documents)} characters.')
print(f'After the split we have {len(docs)} documents as opposed to the original {len(documents)}.')
print(f'Average length among {len(docs)} documents (after split) is {avg_doc_length(docs)} characters.')

Average length among 25 documents loaded is 4131 characters.
After the split we have 151 documents as opposed to the original 25.
Average length among 151 documents (after split) is 699 characters.


We had 4 PDF documents which have been split into smaller chunks.

Now we can see how a sample embedding would look like for one of those chunks.

In [18]:
sample_embedding = np.array(bedrock_embeddings.embed_query(docs[0].page_content))
print("Sample embedding of a document chunk: ", sample_embedding)
print("Size of the embedding: ", sample_embedding.shape)

Sample embedding of a document chunk:  [ 0.71484375 -0.49804688 -0.0014267  ...  0.3125     -0.09130859
 -0.42578125]
Size of the embedding:  (1536,)


Following the very same approach embeddings can be generated for the entire corpus and stored in a vector store.

This can be easily done using SingleStoreDB implementation inside [LangChain](https://python.langchain.com/docs/integrations/vectorstores/singlestoredb) which takes  input the embeddings model and the documents to create the entire vector store.

**⚠️⚠️⚠️ NOTE: it might take few minutes to run the following cell ⚠️⚠️⚠️**

In [19]:
from langchain.vectorstores import SingleStoreDB

vectorstore_s2 = SingleStoreDB.from_documents(
    docs,
    bedrock_embeddings,
)

### Question Answering

Now that we have our vector store in place, we can start asking questions.

In [20]:
query = "How has AWS evolved?"

The first step would be to create an embedding of the query such that it could be compared with the documents

In [21]:
query_embedding = bedrock_embeddings.embed_query("This is a content of the document")
np.array(query_embedding)

array([ 0.75390625, -0.34375   , -0.37890625, ...,  0.34765625,
       -0.734375  , -0.13183594])

We can use this embedding of the query to then fetch relevant documents.
Now our query is represented as embeddings we can do a similarity search of our query against our data store providing us with the most relevant information.

In [22]:
relevant_documents = vectorstore_s2.similarity_search(query)
print(f'{len(relevant_documents)} documents are fetched which are relevant to the query.')
print('----')
for i, rel_doc in enumerate(relevant_documents):
    print_ww(f'## Document {i+1}: {rel_doc.page_content}.......')
    print('---')

4 documents are fetched which are relevant to the query.
----
## Document 1: a significant differentiator), butalso allowed us to arrive at the much more game-
changing offering that AWS is today........
---
## Document 2: This growth also created short-term logistics and cost challenges. We spent Amazon’s
first 25 years building.......
---
## Document 3: In the early days of AWS, people sometimes asked us why compute wouldn’t just be an
undifferentiated.......
---
## Document 4: it’s been. Whatever role Amazon played in the world up to that point became further
magnified as mostphysical venues shut down for long periods of time and people spent their days at
home. This meant thathundreds of millions of people relied on Amazon for PPE, food, clothing, and
various other items thathelped them navigate this unprecedented time. Businesses and governments
also had to shift, practicallyovernight, from working with colleagues and technology on-premises to
working remotely. AWS played amajor r

Now we have the relevant documents, it's time to use the LLM to generate an answer based on these documents. 

We will take our inital prompt, together with our relevant documents which were retreived based on the results of our similarity search. We then by combining these create a prompt that we feed back to the model to get our result. At this point our model should give us highly informed information on how we can change the tire of our specific car as it was outlined in our manual.

LangChain provides an abstraction of how this can be done easily.

### Customisable option
In the above scenario you explored the quick and easy way to get a context-aware answer to your question. Now let's have a look at a more customizable option with the help of [RetrievalQA](https://python.langchain.com/en/latest/modules/chains/index_examples/vector_db_qa.html) where you can customize how the documents fetched should be added to prompt using `chain_type` parameter. Also, if you want to control how many relevant documents should be retrieved then change the `k` parameter in the cell below to see different outputs. In many scenarios you might want to know which were the source documents that the LLM used to generate the answer, you can get those documents in the output using `return_source_documents` which returns the documents that are added to the context of the LLM prompt. `RetrievalQA` also allows you to provide a custom [prompt template](https://python.langchain.com/en/latest/modules/prompts/prompt_templates/getting_started.html) which can be specific to the model.

Note: In this example we are using Anthropic Claude as the LLM under Amazon Bedrock, this particular model performs best if the inputs are provided under `Human:` and the model is requested to generate an output after `Assistant:`. In the cell below you see an example of how to control the prompt such that the LLM stays grounded and doesn't answer outside the context.

In [23]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """

Human: Use the following pieces of context to provide a concise answer to the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}

Assistant:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore_s2.as_retriever(
        search_type="similarity", search_kwargs={"k": 3}
    ),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT},
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [24]:
query = "How did AWS evolve?"
result = qa({"query": query})
print_ww(result['result'])

print(f"\n{result['source_documents']}")

 Based on the provided context, it seems AWS evolved from initially offering basic compute services
to becoming a more expansive and game-changing offering over time. The context indicates this growth
also created some logistics and cost challenges for Amazon.

[Document(page_content='a significant differentiator), butalso allowed us to arrive at the much more game-changing offering that AWS is today.', metadata={'source': 'AMZN-2021-Shareholder-Letter.pdf', 'year': 2021}), Document(page_content='This growth also created short-term logistics and cost challenges. We spent Amazon’s first 25 years building', metadata={'source': 'AMZN-2021-Shareholder-Letter.pdf', 'year': 2021}), Document(page_content='In the early days of AWS, people sometimes asked us why compute wouldn’t just be an undifferentiated', metadata={'source': 'AMZN-2021-Shareholder-Letter.pdf', 'year': 2021})]


In [27]:
query = "Why is Amazon successful?"
result = qa({"query": query})
print_ww(result['result'])

print(f"\n{result['source_documents']}")

 Based on the provided context, it seems Amazon's success stems from investing in and building its
ecommerce and logistics capabilities over many years. The context indicates Amazon grew steadily
over 25 years by building out its ecommerce platform and logistics networks, and it continues making
investments that leverage these core strengths. However, without more complete information I cannot
confidently explain the full reasons behind Amazon's success.

[Document(page_content='This growth also created short-term logistics and cost challenges. We spent Amazon’s first 25 years building', metadata={'source': 'AMZN-2021-Shareholder-Letter.pdf', 'year': 2021}), Document(page_content='Amazon Business is another example of an investment where our ecommerce and logistics capabilities', metadata={'source': 'AMZN-2022-Shareholder-Letter.pdf', 'year': 2022}), Document(page_content='We want to improve workers’ lives beyond pay. Amazon provides every full-time employee with health', metadata={'so

In [28]:
query = "What business challenges has Amazon experienced?"
result = qa({"query": query})
print_ww(result['result'])

print(f"\n{result['source_documents']}")

 Based on the context provided, some of the business challenges Amazon has experienced are:

- Short-term logistics and cost challenges from rapid growth of the company in its early years.

- Rising costs to fulfill and deliver orders from Amazon's warehouses and stores (its "Stores
fulfillment network").

- Rapid growth in its consumer business during the COVID-19 pandemic which likely strained its
fulfillment and delivery capacity.

[Document(page_content='This growth also created short-term logistics and cost challenges. We spent Amazon’s first 25 years building', metadata={'source': 'AMZN-2021-Shareholder-Letter.pdf', 'year': 2021}), Document(page_content='Amazon Business is another example of an investment where our ecommerce and logistics capabilities', metadata={'source': 'AMZN-2022-Shareholder-Letter.pdf', 'year': 2022}), Document(page_content='A critical challenge we’ve continued to tackle is the rising cost to serve in our Stores fulfillment network (i.e.\nthe cost to get a p

In [29]:
query = "How was Amazon impacted by COVID-19?"

result = qa({"query": query})

print_ww(result['result'])

print(f"\n{result['source_documents']}")

 Based on the context provided, it seems Amazon was impacted by COVID-19 in the following ways:

- There was a major increase in demand for Amazon's retail business as people stayed home and relied
on Amazon for supplies, increasing the company's reach and importance.

- AWS saw increased demand as businesses rapidly shifted to remote work and needed cloud
infrastructure. AWS helped many companies adjust and maintain business continuity.

- The growth created logistics and cost challenges for Amazon in the short-term as they worked to
meet increased demand.

- The different trajectories of the retail business and AWS underscored the breadth of Amazon's
business.

In summary, COVID-19 led to surges in demand that demonstrated the critical role Amazon plays for
consumers and businesses, while also surfacing operational challenges as the company scaled to meet
these new demands.

[Document(page_content='it’s been. Whatever role Amazon played in the world up to that point became further ma

## Clean up
Clear the downloaded PDFs

In [30]:
!rm -rf ./RAG_Titan_Embeddings_Claude_data

## Conclusion
Retrieval augmented generation is an important technique that combines the power of large language models with the precision of retrieval methods. By augmenting generation with relevant retrieved examples, the responses we recieved become more coherent, consistent and grounded. You should feel proud of learning this innovative approach. I'm sure the knowledge you've gained will be very useful for building creative and engaging language generation systems. 

In the above implementation of RAG based Question Answering we have explored the following concepts and how to implement them using Amazon Bedrock and it's LangChain integration.

- Loading documents of different kind and generating embeddings to create a vector store
- Retrieving documents to the question
- Preparing a prompt which goes as input to the LLM
- Present an answer in a human friendly manner

### Take-aways
- Leverage various models available under Amazon Bedrock to see alternate outputs
- Explore options such as persistent storage of embeddings and document chunks
- Integration with enterprise data stores

# Thank You